# Пример: кластеризация текстов

## Выборка

In [36]:
from sklearn.datasets import fetch_20newsgroups

train_all = fetch_20newsgroups(subset='train')
print train_all.target_names

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [82]:
simple_dataset = fetch_20newsgroups(
    subset='train', 
    categories=['comp.sys.mac.hardware', 'soc.religion.christian', 'rec.sport.hockey'],
)

In [83]:
print simple_dataset.data[0]

From: erik@cheshire.oxy.edu (Erik Adams)
Subject: HELP!!  My Macintosh "luggable" has lines on its screen!
Organization: Occidental College, Los Angeles, CA 90041 USA.
Distribution: comp
Lines: 20

Okay, I don't use it very much, but I would like for it to keep working
correctly, at least as long as Apple continues to make System software
that will run on it, if slowly :-)

Here is the problem:  When the screen is tilted too far back, vertical
lines appear on the screen.  They are every 10 pixels or so, and seem
to be affected somewhat by opening windows and pulling down menus.
It looks to a semi-technical person like there is a loose connection
between the screen and the rest of the computer.

I am open to suggestions that do not involve buying a new computer,
or taking this one to the shop.  I would also like to not have
to buy one of Larry Pina's books.  I like Larry, but I'm not sure
I feel strongly enough about the computer to buy a service manual
for it.

On a related note:  what

In [84]:
simple_dataset.target

array([0, 0, 1, ..., 0, 1, 2])

In [85]:
print simple_dataset.data[-1]

From: dlecoint@garnet.acns.fsu.edu (Darius_Lecointe)
Subject: Re: Sabbath Admissions 5of5
Organization: Florida State University
Lines: 21

I find it interesting that cls never answered any of the questions posed. 
Then he goes on the make statements which make me shudder.  He has
established a two-tiered God.  One set of rules for the Jews (his people)
and another set for the saved Gentiles (his people).  Why would God
discriminate?  Does the Jew who accepts Jesus now have to live under the
Gentile rules.

God has one set of rules for all his people.  Paul was never against the
law.  In fact he says repeatedly that faith establishes rather that annuls
the law.  Paul's point is germane to both Jews and Greeks.  The Law can
never be used as an instrument of salvation.  And please do not combine
the ceremonial and moral laws in one.

In Matt 5:14-19 Christ plainly says what He came to do and you say He was
only saying that for the Jews's benefit.  Your Christ must be a
politician, speaki

In [86]:
print simple_dataset.data[-2]

From: scialdone@nssdca.gsfc.nasa.gov (John Scialdone)
Subject: CUT Vukota and Pilon!!!
News-Software: VAX/VMS VNEWS 1.41    
Organization: NASA - Goddard Space Flight Center
Lines: 32

I have been to all 3 Isles/Caps tilts at the Crap Centre this year, all Isles
wins and there is no justification for Vukota and Pilon to play for the Isles.
Vukota is absolutely the worst puck handler in the world!! He couldn't hit a
bull in the ass with a banjo!! Al must remember a few years back when Mick 
scored 3 goals in one period against the Caps in a 5-3 Isles win. I was there
and was astonished as was the rest of the crowd. Wake-up Al!!! Years later he's
gotten worse. He's a cheap shot artist and always ends up getting
stupid/senseless penalties. I think he would make a good police officier!!!

As for Pilon, he can't carry the puck out to center ice by himself. He either
makes a bad pass resulting in a turnover, or he attempts to bring the puck 
towards the neutral zone and skates right into an 

In [88]:
print len(simple_dataset.data)

1777


## Признаки

In [89]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=500, min_df=10)
matrix = vectorizer.fit_transform(simple_dataset.data)

In [90]:
print matrix.shape

(1777, 3767)


In [129]:
import itertools
def validate_with_mappings(preds, target):
    permutations = itertools.permutations([0, 1, 2])
    for a, b, c in permutations:
        mapping = {2 : a, 1: b, 0: c}
        mapped_preds = [mapping[pred] for pred in preds]
        print float(sum(mapped_preds == target)) / len(target)

## Аггломеративная кластеризация (neighbour joining)

In [96]:
from sklearn.cluster.hierarchical import AgglomerativeClustering

model = AgglomerativeClustering(n_clusters=3, affinity='cosine', linkage='complete')
preds = model.fit_predict(matrix.toarray())

In [97]:
validate_with_mappings(preds, simple_dataset.target)

0.325267304446
0.327518289252
0.347214406303
0.359032076533
0.315700619021
0.325267304446


In [98]:
from collections import Counter
Counter(preds)

Counter({0: 1717, 2: 39, 1: 21})

## KMeans

In [99]:
from sklearn.cluster import KMeans

model = KMeans(n_clusters=3, random_state=1)
preds = model.fit_predict(matrix.toarray())
print preds

[0 0 2 ..., 0 2 1]


In [100]:
validate_with_mappings(preds, simple_dataset.target)

0.0298255486775
0.326392796849
0.349465391109
0.951603826674
0.0185706246483
0.324141812043


In [101]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
clf = LogisticRegression()
print cross_val_score(clf, matrix, simple_dataset.target).mean()


0.985360318588


In [106]:
vectorizer.get_feature_names()[877]

u'connect'

## Более сложная выборка

In [111]:
dataset = fetch_20newsgroups(
    subset='train', 
    categories=['comp.sys.mac.hardware', 'comp.os.ms-windows.misc', 'comp.graphics'],
)

In [130]:
matrix = vectorizer.fit_transform(dataset.data)
model = KMeans(n_clusters=3, random_state=42)
preds = model.fit_predict(matrix.toarray())

In [131]:
import itertools
def validate_with_mappings2(preds, target):
    clusters = len(set(preds))
    mapping = {
        i: i
        for i in xrange(clusters)
    }
    for i in xrange(clusters):
        best_j = -1
        best_val = -1
        for j in xrange(clusters):
            mapping[i] = j
            mapped_preds = [mapping[pred] for pred in preds]
            acc = float(sum(mapped_preds == target)) / len(target)
            if acc > best_val:
                best_j = j
                best_val = acc
        mapping[i] = best_j
    return mapping, best_val

In [132]:
validate_with_mappings2(preds, dataset.target)

({0: 2, 1: 1, 2: 0}, 0.7387335995436395)

In [133]:
validate_with_mappings(preds, dataset.target)

0.738733599544
0.306902452938
0.38961779806
0.133485453508
0.127780946948
0.303479749002


In [135]:
for size in xrange(1, 10):
    model = KMeans(n_clusters=size, random_state=42)
    preds = model.fit_predict(matrix.toarray())
    _, acc = validate_with_mappings2(preds, dataset.target)
    print 'Size: {}\tAccuracy: {}'.format(size, acc)

Size: 1	Accuracy: 0.333143183115
Size: 2	Accuracy: 0.495151169424
Size: 3	Accuracy: 0.738733599544
Size: 4	Accuracy: 0.779235596121
Size: 5	Accuracy: 0.731317741015
Size: 6	Accuracy: 0.80091272105
Size: 7	Accuracy: 0.738163148888
Size: 8	Accuracy: 0.712492869367
Size: 9	Accuracy: 0.717056474615


In [76]:
clf = LogisticRegression()
print cross_val_score(clf, matrix, dataset.target).mean()

0.917279226713


## SVD + KMeans

In [77]:
from sklearn.decomposition import TruncatedSVD

model = KMeans(n_clusters=3, random_state=42)
svd = TruncatedSVD(n_components=1000, random_state=123)
features = svd.fit_transform(matrix)
preds = model.fit_predict(features)

In [78]:
validate_with_mappings(preds, dataset.target)

0.406731317741
0.0889903023388
0.793496862521
0.297204791786
0.296063890473
0.11751283514


In [79]:
model = KMeans(n_clusters=3, random_state=42)
svd = TruncatedSVD(n_components=200, random_state=123)
features = svd.fit_transform(matrix)
preds = model.fit_predict(features)

In [80]:
validate_with_mappings(preds, dataset.target)

0.0992584141472
0.325727324586
0.294352538505
0.106103822019
0.794637763833
0.379920136908


In [81]:
model = KMeans(n_clusters=3, random_state=42)
svd = TruncatedSVD(n_components=200, random_state=321)
features = svd.fit_transform(matrix)
preds = model.fit_predict(features)
validate_with_mappings(preds, dataset.target)

0.286936679977
0.154592127781
0.110667427268
0.29948659441
0.413576725613
0.734740444952


## Итоги

1. Получили интерпретируемый результат на обеих выборках
1. Реальность, однако, намного более жестока
1. Попробовали использовать AgglomerativeClustering и KMeans